Xgboost

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
train_data = pd.read_csv("featuresets/3/train_data_one_hot.csv",index_col=0)
test_data = pd.read_csv("featuresets/3/test_data_one_hot.csv",index_col=0)
submissions = pd.read_csv("data/sample_submission.csv")

In [3]:
train_data.head()

,cat89,cat90,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,...,cat86 = C,cat86 = D,cat87 = A,cat87 = B,cat87 = C,cat87 = D,cat88 = A,cat88 = B,cat88 = D,cat88 = E
0,0,0,0,0,3,1,2,4,0,2,...,0,1,0,1,0,0,1,0,0,0
1,0,0,0,0,3,3,2,4,4,3,...,0,1,0,1,0,0,1,0,0,0
2,0,0,0,0,3,3,2,4,4,0,...,0,0,0,1,0,0,1,0,0,0
3,0,0,0,0,3,3,2,4,4,3,...,0,1,0,1,0,0,1,0,0,0
4,0,0,1,5,3,1,3,4,4,0,...,0,0,0,0,1,0,1,0,0,0


In [4]:
x_train = train_data.drop("loss",axis=1)
y = np.log(train_data["loss"])
x_test = test_data

In [5]:
def write_submission_file(predictions,name):
    if not os.path.exists("results"):
        os.makedirs("results")
    submissions.iloc[:,1] = predictions
    submissions.to_csv('results/' + name + '.csv', index=None)

In [6]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error,mean_absolute_error

/home/achal/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
dtrain = xgb.DMatrix(x_train, label=y)
dtest = xgb.DMatrix(x_test)

In [8]:
xgb_params = {
    'seed': 0,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.070,
    'objective': 'reg:linear',
    'max_depth': 6,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'eval_metric': 'mae',
}

In [9]:
def xg_eval_mse(yhat, dtrain):
    y = dtrain.get_label()
    return('mae', mean_absolute_error(np.exp(y), np.exp(yhat)))

In [10]:
res = xgb.cv(xgb_params, dtrain, num_boost_round=850, nfold=4, seed=20, stratified=False,early_stopping_rounds=15, verbose_eval=10, show_stdv=True, feval=xg_eval_mse, maximize=False)

[0]	train-mae:3034.64+4.97266	test-mae:3034.64+14.9178
[10]	train-mae:2948.3+4.93866	test-mae:2948.31+14.8059
[20]	train-mae:2545.35+4.54527	test-mae:2545.65+14.1612
[30]	train-mae:1938.82+3.27138	test-mae:1940.97+12.8308
[40]	train-mae:1507.94+2.61441	test-mae:1514.54+10.4707
[50]	train-mae:1306.63+1.89889	test-mae:1317.97+8.70301
[60]	train-mae:1223.43+2.18555	test-mae:1238.6+7.13049
[70]	train-mae:1186.34+2.27379	test-mae:1204.84+6.72341
[80]	train-mae:1166.77+2.2542	test-mae:1188.17+6.49831
[90]	train-mae:1154.72+2.24705	test-mae:1178.75+6.55914
[100]	train-mae:1146.22+2.67807	test-mae:1172.62+6.3424
[110]	train-mae:1139.76+2.66441	test-mae:1168.23+6.4902
[120]	train-mae:1134.36+2.72227	test-mae:1165.22+6.37027
[130]	train-mae:1129.57+2.84836	test-mae:1162.59+6.30863
[140]	train-mae:1125.4+2.58543	test-mae:1160.46+6.53319
[150]	train-mae:1121.66+2.4977	test-mae:1158.78+6.50598
[160]	train-mae:1117.98+2.4499	test-mae:1157.27+6.47945
[170]	train-mae:1114.63+2.3864	test-mae:1155.85+6.

In [11]:
best_nrounds = res.shape[0] - 1
cv_mean = res.iloc[-1, 0]
cv_std = res.iloc[-1, 1]
print('CV-Mean: {0}+{1}'.format(cv_mean, cv_std))

CV-Mean: 1146.3098755+6.517141641891779


In [12]:
gbdt = xgb.train(xgb_params, dtrain, best_nrounds)

In [13]:
write_submission_file(np.exp(gbdt.predict(dtest)),"xgb_tuned")